In [1]:
import csv
from tabulate import tabulate
import os
from colorama import Fore, Back, Style
import sys

In [2]:
""" 
DATA_LIST = [] 
will be used to 'def command(value) = "close",
when closing the program,
to print all the new products inserted in the dataset before closing

DATA_LIST_SALES will be populated with the products sold

"""
global DATA_LIST

DATA_LIST = []
DATA_LIST_SALES = []

In [3]:
# Custom function to color text
def print_colored(text, color="white", bg_color=None, end="\n"):
    # Text Color Dictionary
    color_dict = {
        'red': Fore.RED,
        'red_bold': Style.BRIGHT + Fore.RED,
        'blue': Fore.BLUE,
        'blue_bold': Style.BRIGHT + Fore.BLUE,
        'white': '\033[97m',  # Bianco puro (ANSI)
        'white_bold': Style.BRIGHT + '\033[97m',  
        'black': Fore.BLACK, 
        'black_bold': Style.BRIGHT + Fore.BLACK,
        'green': Fore.GREEN, 
        'green_bold': Style.BRIGHT + Fore.GREEN
    }

    # Background Color Dictionary
    bg_color_dict = {
        'black': Back.BLACK,
        'blue': Back.BLUE,
        'white': Back.WHITE
    }

    color_code = color_dict.get(color.lower(), '\033[97m') 
    bg_color_code = bg_color_dict.get(bg_color.lower(), '') if bg_color else ''
    
    print(f"{color_code}{bg_color_code}{text}{Style.RESET_ALL}", end=end)

In [4]:
""" 
Creating the header of the dataset "data_list"

"""

fields = ["Prodotto", "Quantità", "Prezzo di acquisto", "Prezzo di vendita", "Vendita registrata"]


In [5]:
""" 
Initial creation of the dataset "data_list"

"""


data_list = [
    {"Prodotto": "latte di soia", "Quantità": 20, "Prezzo di acquisto": 0.80, "Prezzo di vendita": 1.40, "Vendita registrata": 0},
    {"Prodotto": "tofu", "Quantità": 10, "Prezzo di acquisto": 2.20, "Prezzo di vendita": 4.19, "Vendita registrata": 0},
    {"Prodotto": "seitan", "Quantità": 5, "Prezzo di acquisto": 3, "Prezzo di vendita": 5.49, "Vendita registrata": 0},
    {"Prodotto": "crema di banana", "Quantità": 35, "Prezzo di acquisto": 10, "Prezzo di vendita": 15, "Vendita registrata": 0},
    {"Prodotto": "avena in polvere", "Quantità": 17, "Prezzo di acquisto": 13.75, "Prezzo di vendita": 20.50, "Vendita registrata": 0},
    {"Prodotto": "proteine vegane", "Quantità": 7, "Prezzo di acquisto": 17.50, "Prezzo di vendita": 24.50, "Vendita registrata": 0},
    {"Prodotto": "proteine di piselli", "Quantità": 34, "Prezzo di acquisto": 8, "Prezzo di vendita": 12, "Vendita registrata": 0},
    {"Prodotto": "barretta proteica", "Quantità": 14, "Prezzo di acquisto": 1.40, "Prezzo di vendita": 2.50, "Vendita registrata": 0},
    {"Prodotto": "crema di mandorle", "Quantità": 5, "Prezzo di acquisto": 3.80, "Prezzo di vendita": 6.50, "Vendita registrata": 0},
    {"Prodotto": "porridge mela e cannella", "Quantità": 8, "Prezzo di acquisto": 9.40, "Prezzo di vendita": 12.50, "Vendita registrata": 0},
    {"Prodotto": "crema di nocciole proteica", "Quantità": 27, "Prezzo di acquisto": 9.40, "Prezzo di vendita": 13.50, "Vendita registrata": 0}
]

In [6]:
"""
This function lists the available commands of the management software


"""

def explain_commands():
    print_colored(f"\nI comandi disponibili sono i seguenti:", "blue_bold")
    print_colored("aggiungi", color="blue", bg_color="white", end="")
    print(": aggiungi un prodotto al magazzino"),
    
    print_colored("elenca", color="blue", bg_color="white", end="")
    print(": elenca i prodotti in magazzino"),
    
    print_colored("vendita", color="blue", bg_color="white", end="")
    print(": registra una vendita effettuata"),
    
    print_colored("profitti", color="blue", bg_color="white", end="")
    print(": mostra i profitti totali"),
    
    print_colored("aiuto", color="blue", bg_color="white", end="")
    print(": mostra i possibili comandi"),
    
    print_colored("chiudi", color="blue", bg_color="white", end="")
    print(": esci dal programma"),

    print_colored("\nSe digiti 'chiudi' per terminare il programma,", "white_bold", end="")
    print_colored(" i dati non ancora salvati andranno persi", "red_bold")

In [7]:
"""
Reads CSV and returns a list of dictionaries
with keys matching the file header
If the file does not exist, returns an empty list
    
"""

def read_csv(data_list): 
    if not os.path.exists(data_list):
        return []
    
    with open(data_list, mode="r", newline="", encoding="utf-8") as f:
        reader = csv.DictReader(f, delimiter=",")

        data = list(reader) # Converts the reader object into a list of dictionaries

    return data       

In [8]:
"""
Create CSV file, update store dataset by adding new products and updating existing ones

"""

def create_add_update_df (name_df, data_list): 
    
    current_data = read_csv(name_df)

     # For each new product in data_list, check if it already exists
    for new_product in data_list:
        found = False
        for record in current_data:
            # Case insensitive comparison on the "Product" field and remove any spaces at the beginning and end
            if record["Prodotto"].strip().lower() == new_product["Prodotto"].strip().lower():
                # Update quantity
                record["Quantità"] = int(record["Quantità"]) + int(new_product["Quantità"])
                # Update prices only if they are present in the new product
                if "Prezzo di acquisto" in new_product:
                    record["Prezzo di acquisto"] = new_product["Prezzo di acquisto"]
                if "Prezzo di vendita" in new_product:
                    record["Prezzo di vendita"] = new_product["Prezzo di vendita"]
                found = True
                break

        if not found:
            # Adds new product if it does not exist
            current_data.append(new_product)

     # Sort records alphabetically by "Product" field
    current_data = sorted(current_data, key=lambda record: record["Prodotto"].strip().lower())
    
    with open(name_df, mode="w", newline="", encoding="utf-8") as f: 
        writer = csv.DictWriter(f, fields, delimiter=",") 

        writer.writeheader()
        
        writer.writerows(current_data)

In [9]:
def check_product_exists(product):
    current_data = read_csv(name_df)
    for record in current_data:
        if record["Prodotto"].strip().lower() == product.strip().lower():
            return True
    return False

In [10]:
"""
If the product already exists in the CSV file, update only the "Quantity" field

"""

def update_quantity_for_existing(product, name_df):
    current_data = read_csv(name_df)
    exists = False
    for record in current_data:
        if record["Prodotto"].strip().lower() == product["Prodotto"].strip().lower():
            record["Quantità"] = int(record["Quantità"]) + int(product["Quantità"])
            exists = True
            break

In [11]:
def commands(value):
    global DATA_LIST
    """
    The value of 'value' will be compared to the content of input() you typed

    """
     
    # List the products in the database
    if value.lower() == "elenca": 
        print_colored("\nIl tuo magazzino contiene i seguenti prodotti:", "blue_bold")

        # View content in tabular form
        content = read_csv(name_df)
        print(tabulate(content, headers="keys", tablefmt="pretty"))

    # Logs the user out of the program
    if value.lower() == "chiudi":
        if not DATA_LIST:
            print_colored("Programma terminato per richiesta dell'utente", "red_bold")
            sys.exit()

In [12]:
def get_product_from_terminal(product_count):
    global DATA_LIST
    
    """
    This function collects the values ​​entered in the input during 'add' mode

    I need to insert 4 'print()' instead of '\n' escape characters in 'f"NEW INSERT - N.'
    otherwise the background color 'bg_color="white"' will color all 4 blank lines in the terminal

    """

    print()
    print()
    print()
    print()
    print_colored(f"NUOVO INSERIMENTO - N.{product_count}", color="blue_bold", bg_color="white")
    print_colored("\nCompila tutti i campi richiesti", color="blue_bold")
    print_colored("\nLascia il campo vuoto e Premi ", "blue_bold", end="")
    print_colored("INVIO", "blue_bold", "white", end="")
    print_colored(" per terminare", "blue_bold")
    print_colored("I dati non ancora salvati andranno persi", "red_bold")
    
    print("\nNome del prodotto: ", end="")
    product = input().strip().lower()

    # If the input is empty and the user clicks ENTER, a new command is requested.
    if not product:
        print_colored("Sei uscito dalla modalità aggiungi\n", "red_bold")
        if not DATA_LIST:
            print_colored("Nessun prodotto aggiunto in magazzino", "red_bold", "white")
        return None  

    # Check if the product already exists in the CSV
    current_data = read_csv(name_df)
    exists = False
    for record in current_data:
        if record["Prodotto"].strip().lower() == product:
            exists = True
            break

    if exists:
        # If the product exists, only the quantity is added
        print_colored("\nIl prodotto esiste già. Inserisci solo la quantità da aggiungere:", "blue_bold")
        
        while True:
            quantity_input = input("Quantità: ").strip()
            
            match quantity_input:
                # If the input is empty and the user clicks ENTER, a new command is requested
                case _ if not quantity_input:
                    print_colored("Sei uscito dalla modalità aggiungi\n", "red_bold")
                    if not DATA_LIST:
                        print_colored("Nessun prodotto aggiunto in magazzino", "red_bold", "white")
                    return None
                
                # If the input contains at least one letter
                case _ if any(c.isalpha() for c in quantity_input):
                    print_colored("Non inserire caratteri alfabetici", "red_bold")
                    continue  # Ripete il ciclo
                
                # If the input contains commas and numbers
                case _ if ("," in quantity_input and any(c.isdigit() for c in quantity_input)):
                    print_colored("Non inserire caratteri alfabetici e inserisci dei numeri interi", "red_bold")
                    continue  # Ripete il ciclo

                # If the input contains no letters
                case _:
                    try:
                        quantity = int(quantity_input)
                    except ValueError:
                        print_colored("Inserisci dei numeri interi.", "red_bold")
                        continue
                    if quantity >= 0:
                        break
                    else:
                        print_colored("Non puoi inserire valori negativi", "red_bold")
                        continue



        new_product = {
            "Prodotto": product,
            "Quantità": quantity
        }

        # CSV Update
        update_quantity_for_existing(new_product, name_df)
        return new_product

    else:
        # If the product does not exist, ask for all the data: quantity, purchase price and selling price
        while True:
            quantity_input = input("Quantità: ").strip()
            
            match quantity_input:
                    # If the input is empty and the user clicks ENTER, a new command is requested.
                    case _ if not quantity_input:
                        print_colored("Sei uscito dalla modalità aggiungi\n", "red_bold")
                        if not DATA_LIST:
                            print_colored("Nessun prodotto aggiunto in magazzino", "red_bold", "white")
                        return None
                    
                    # If the input contains at least one letter
                    case _ if any(c.isalpha() for c in quantity_input):
                        print_colored("Non inserire caratteri alfabetici", "red_bold")
                        continue  # Ripete il ciclo
                    
                    # If the input contains commas and numbers
                    case _ if ("," in quantity_input and any(c.isdigit() for c in quantity_input)):
                        print_colored("Non inserire caratteri alfabetici e inserisci dei numeri interi", "red_bold")
                        continue  # Ripete il ciclo
    
                    # If the input contains no letters
                    case _:
                        try:
                            quantity = int(quantity_input)
                        except ValueError:
                            print_colored("Inserisci un numero intero, non un decimale", "red_bold")
                            continue
                        if quantity >= 0:
                            break
                        else:
                            print_colored("Non puoi inserire valori negativi", "red_bold")
                            continue
 

        # Entering the purchase price
        while True:
            purchase_price_input = input("Prezzo di acquisto (usa il punto come separatore decimale): ").strip()
            
            match purchase_price_input:
            
                # If the input is empty and the user clicks ENTER, a new command is requested
                case _ if not purchase_price_input:
                    print_colored("Sei uscito dalla modalità aggiungi\n", "red_bold")
                    if not DATA_LIST:
                        print_colored("Nessun prodotto aggiunto in magazzino", "red_bold", "white")
                    return None
                    continue
                
                # If the input contains both a comma, at least one letter and at least one digit
                case _ if ("," in purchase_price_input and any(c.isalpha() for c in purchase_price_input) and any(c.isdigit() for c in purchase_price_input)):
                    print_colored("Non inserire caratteri alfabetici e usa il punto (.) come separatore decimale, non la virgola (,)", "red_bold")
                    continue
                
                # If the input contains at least one letter
                case _ if any(c.isalpha() for c in purchase_price_input):
                    print_colored("Non inserire caratteri alfabetici", "red_bold")
                    continue
                
                # If the input contains a comma, even if there are no letters
                case _ if "," in purchase_price_input:
                    print_colored("Usa il punto (.) come separatore decimale, non la virgola (,)", "red_bold")
                    continue

                # If the input contains no letters
                case _:
                    purchase_price_float = float(purchase_price_input)
                    if purchase_price_float >= 0:
                        break  # If the conversion is successful, it exits the cycle
                    else:
                        print_colored("Non puoi inserire valori negativi", "red_bold")
                        continue



        # Entering the purchase price
        while True:
            selling_price_input = input("Prezzo di vendita (usa il punto come separatore decimale): ").strip()
            
            match selling_price_input:
            
                # If the input is empty and the user clicks ENTER, a new command is requested.
                case _ if not selling_price_input:
                    print_colored("Sei uscito dalla modalità aggiungi\n", "red_bold")
                    if not DATA_LIST:
                        print_colored("Nessun prodotto aggiunto in magazzino", "red_bold", "white")
                    return None
                    continue 
                
                # If the input contains both a comma, at least one letter and at least one digit
                case _ if ("," in selling_price_input and any(c.isalpha() for c in selling_price_input) and any(c.isdigit() for c in selling_price_input)):
                    print_colored("Non inserire caratteri alfabetici e usa il punto (.) come separatore decimale, non la virgola (,)", "red_bold")
                    continue
                
                # If the input contains at least one letter
                case _ if any(c.isalpha() for c in selling_price_input):
                    print_colored("Non inserire caratteri alfabetici", "red_bold")
                    continue
                
                # If the input contains a comma, even if there are no letters
                case _ if "," in selling_price_input:
                    print_colored("Usa il punto (.) come separatore decimale, non la virgola (,)", "red_bold")
                    continue

                # If the input contains no letters
                case _:
                    selling_price_float = float(selling_price_input)
                    if selling_price_float >= 0:
                        break
                    else:
                        print_colored("Non puoi inserire valori negativi", "red_bold")
                        continue

        new_product = {
            "Prodotto": product,
            "Quantità": quantity,
            "Prezzo di acquisto": purchase_price_float,
            "Prezzo di vendita": selling_price_float
        }
        return new_product


In [13]:
def get_product_from_terminal_for_sale():
    global DATA_LIST_SALES

    """
    This function collects the values ​​entered in the input during the 'sale' mode

    """
    
    while True:  
        print_colored("\nLascia il campo vuoto e Premi ", "blue_bold", end="")
        print_colored("INVIO", "blue_bold", "white", end="")
        print_colored(" per terminare", "blue_bold")
        product = input("\nNome del prodotto:\n").strip().lower()
        if not product:
            # If the input is empty and the user clicks ENTER and a new command is requested
            if not DATA_LIST_SALES:
                print_colored("Sei uscito dalla modalità vendite\n", "red_bold")
                print_colored("Nessun prodotto registrato nelle vendite", "red_bold", "white")
                    
            if DATA_LIST_SALES:
                print_colored("Sei uscito dalla modalità vendite\n", "red_bold")
                data_list_sales(DATA_LIST_SALES)
            return None


            # Check if the product exists in the CSV
            data = read_csv(data_list)
            product_exists = any(record["Prodotto"].strip().lower() == product for record in data)
            if not product_exists:
                print_colored("Prodotto non presente in magazzino. La vendita non può essere registrata", "red_bold")
                return None

        # Inserting N. Product Quantity
        while True:
            quantity_input = input("Quantità: ").strip()
            match quantity_input:
                case _ if not quantity_input:
                    if not DATA_LIST_SALES:
                        print_colored("Sei uscito dalla modalità vendite\n", "red_bold")
                        print_colored("Nessun prodotto registrato nelle vendite", "red_bold", "white")
                    
                    if DATA_LIST_SALES:
                        print_colored("Sei uscito dalla modalità vendite\n", "red_bold")
                        data_list_sales(DATA_LIST_SALES)
                        
                    return None
                case _ if any(c.isalpha() for c in quantity_input):
                    print_colored("Non inserire caratteri alfabetici per la quantità", "red_bold")
                    continue
                
                case _ if ("," in quantity_input and any(c.isdigit() for c in quantity_input)):
                    print_colored("Non inserire virgole. Inserisci un numero intero per la quantità", "red_bold")
                    continue
                
                case _: # If the input contains no letters check if the value is positive
                        quantity = int(quantity_input)
                        if quantity > 0:
                            break  
                        print_colored("Non puoi inserire valori negativi", "red_bold")
 

        # Builds the dictionary with product data
        product_sale = {
            "Prodotto": product,
            "Quantità venduta": quantity
        }
        DATA_LIST_SALES.append(product_sale)
        update_quantity_after_sale(product_sale, name_df)
        


        
#------------------------------------------------------------------------------
# Request to record any new sale in the dataset
#------------------------------------------------------------------------------        
        
        
        
        """
        After recording a sale in the warehouse the user is asked if he wants to record another sale

        """
        
        print_colored("\nAggiungere un altro prodotto? (digita 's' per continuare, altrimenti", "blue_bold", end="")
        print_colored("\nLascia il campo vuoto e Premi ", "blue_bold", end="")
        print_colored("INVIO", "blue_bold", "white", end="")
        print_colored(" per terminare):", "blue_bold", end="")
        add_more = input().strip().lower()
        
        if add_more != "s":
            return None
    
    # After the loop the function to print the summary is called
    data_list_sales(DATA_LIST_SALES)
    
    return DATA_LIST_SALES

    
def data_list_sales(DATA_LIST_SALES):
    
    if DATA_LIST_SALES:
        sales = []
        grand_total = 0.0  # Global counter for total sales

        print()
        print_colored("VENDITA REGISTRATA", "blue_bold", "white")
        for product_data in DATA_LIST_SALES:
            
            # Retrieving the sales price from CSV for the current product
            price = get_selling_price(product_data['Prodotto'], name_df)
            
            if price is not None:
                # Formatting price as float
                str_price = f"{price:.2f} €"
                try:
                    quantity_sold_input = float(product_data['Quantità venduta'])# The amount entered by the user
                except Exception:
                    print_colored("Errore nella conversione della quantità venduta", "red_bold")
                    continue
            else:
                str_price = print_colored("Prodotto non presente in magazzino. La vendita non può essere registrata", "red_bold")
                quantity_sold_input = float(product_data.get('Quantità venduta', 0))
        
            # Calculating Total Sales
            total = price * quantity_sold_input if price is not None else 0.0
            grand_total += total
           
                
        
            # Formatted string for the current product
            sale_string = f"- {quantity_sold_input:.0f} X {product_data['Prodotto']}: {str_price}"
            sales.append(sale_string)
        
        for s in sales:
            print_colored(s, "blue_bold")
        
        print_colored(f"\nTotale: {grand_total:.2f} €", "blue_bold")
        
    DATA_LIST_SALES.clear() 
    
    return DATA_LIST_SALES

In [14]:
def update_quantity_after_sale(product_data, file_path):
    """
    Updates the CSV by subtracting the quantity sold from the quantity available
    and increments the "Sale Recorded" field with the quantity sold only if
    the sale is recorded, i.e., if there is still enough stock
    
    If the product in stock is already out of stock (Quantity == 0) or if the quantity you would like to sell
    exceeds the available quantity, the function displays a suitable 'print()'
    
    """
    data = read_csv(file_path)
    product_found = False

    for record in data:
        if record["Prodotto"].strip().lower() == product_data["Prodotto"].strip().lower():
            try:
                current_quantity = int(record["Quantità"])
                quantity_sold = int(product_data["Quantità venduta"])
            except ValueError:
                print_colored("Errore di conversione nella quantità", "red_bold")
                return
            
            # If the product is already out of stock
            if current_quantity == 0:
                print_colored("Il prodotto in magazzino è esaurito. La vendita non può essere registrata", "red_bold")
                product_data["Quantità venduta"] = 0
                product_found = True
                break
            
            # If the quantity you would like to sell exceeds the quantity available
            if quantity_sold > current_quantity:
                print_colored("Non ci sono abbastanza prodotti in magazzino per completare la vendita", "red_bold")
                print_colored(f"Quantità disponibile: {current_quantity}", "red_bold")
                product_data["Quantità venduta"] = 0
                product_found = True
                break
            
            # If the quantity sold runs out of product
            if current_quantity == quantity_sold:
                record["Quantità"] = 0
                # Update the 'Sale Recorded' field
                # If the field does not exist, assume 0
                previous_registered_sale = int(record.get("Vendita registrata") or 0)

                new_sale_registered = previous_registered_sale + quantity_sold
                record["Vendita registrata"] = new_sale_registered
                product_found = True
                print_colored("Vendita registrata ma il prodotto in magazzino è terminato", "red_bold")
                break
            
            # If the quantity sold does not exhaust the product
            else:
                new_remaining_quantity = current_quantity - quantity_sold
                record["Quantità"] = new_remaining_quantity
                previous_registered_sale = int(record.get("Vendita registrata", 0) or 0)

                new_sale_registered = previous_registered_sale + quantity_sold
                record["Vendita registrata"] = new_sale_registered
                product_found = True
                print_colored("Vendita registrata", "blue_bold", "white")
                break

    if not product_found:
        print_colored("Prodotto non presente in magazzino. La vendita non può essere registrata", "red_bold")
        return

    # Update CSV
    with open(file_path, mode="w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=fields, delimiter=",")
        writer.writeheader()
        writer.writerows(data)

In [15]:
def get_selling_price(product_name, file_path):
    """
    This function searches the CSV for the product whose name (field "Product")
    matches product_name (case-insensitive comparison)
    and returns the value associated with the key "Sale Price" as a float
    
    If the product is not found, it returns None
    
    """
    data = read_csv(file_path)
    for record in data:
        if record["Prodotto"].strip().lower() == product_name.strip().lower():
            try:
                return float(record["Prezzo di vendita"])
            except ValueError:
                return None
    return None


In [16]:
if __name__ == "__main__":
    
    explain_commands()
    
    name_df = "magazzino_prodotti.csv"
    
    # Initialization of the product counter and the list of products entered in 'input' during 'add' mode
    product_count = 0 
    data_list_updated = []  # Global list for products inserted during the session

    print_colored("\n\nInserisci i dati dei prodotti da aggiungere o aggiornare nel magazzino", "white_bold")

    while True:
        print_colored("\n\n\n\nInserisci un comando (aggiungi, elenca, vendita, profitti, aiuto, chiudi)\n", "blue_bold")
        value = input("").strip().lower()
        commands(value)
        
        if value == "aiuto":
            explain_commands()
            continue  # Return to request a new command
        
        elif value == "elenca": # You need to add the check for the 'list' command with 'pass' too
            pass                # otherwise 'else' will also be executed after 'elenca'
           
        
        elif value.lower() == "aggiungi":
            product_count = 0
            print_colored("Comando 'aggiungi' selezionato", "blue_bold")
            data_list_updated = [] 
            
            while True:  # Cycle to repeatedly insert new products
                product_count += 1
                new_product = get_product_from_terminal(product_count)
                
                # If the function does not return a product, the insertion ends
                if new_product is None:
                    # If there are registered products, build the summary
                    if DATA_LIST:
                        products_added = []
                        for p in DATA_LIST:
                            products_added.append(f"{p['Quantità']} X {p['Prodotto']}")
                        if len(products_added) == 1:
                            print_colored("Hai aggiunto il seguente prodotto:", "red_bold")
                        elif len(products_added) >= 2:
                            print_colored("Hai aggiunto i seguenti prodotti:", "red_bold")
                        for a in products_added:
                            print_colored(a, "blue_bold")
                        
                        DATA_LIST.clear()# Clear the global list
                    else:
                        pass
                    break 
                
                # If the user has entered a product, it is added to the update list
                data_list_updated.append(new_product)
                
                if check_product_exists(new_product["Prodotto"]):
                    # Product already exists: print a message indicating that you are updating the quantity
                    print_colored("\nHai aggiornato la quantità per il prodotto esistente:", "red_bold")
                    print_colored(f"Prodotto: {new_product['Prodotto']}", "blue_bold")
                    print_colored(f"Quantità aggiunta: {new_product['Quantità']}", "blue_bold")
                else:
                    # The product is new: print the complete message with all the details
                    print_colored("\nHai aggiunto il seguente prodotto:", "red_bold")
                    print_colored(f"Prodotto: {new_product['Prodotto']}", "blue_bold")
                    print_colored(f"Quantità: {new_product['Quantità']}", "blue_bold")
                    print_colored(f"Prezzo di acquisto: {new_product['Prezzo di acquisto']}", "blue_bold")
                    print_colored(f"Prezzo di vendita: {new_product['Prezzo di vendita']}", "blue_bold")
                    print_colored(f"AGGIUNTO: {new_product['Quantità']} X {new_product['Prodotto']}", "blue_bold")


                # The CSV is updated with the new product
                create_add_update_df(name_df, [new_product])

                
                # Update the global variable DATA_LIST with the inserted products
                DATA_LIST = data_list_updated
                

        elif value == "profitti":
            data = read_csv(name_df)  # Read the updated CSV
            profit_total = 0.0
            print_colored("CALCOLO DEI PROFITTI:\n", "red_bold")
            for record in data:
                try:
                    purchase_price = float(record["Prezzo di acquisto"])
                    selling_price = float(record["Prezzo di vendita"])
                   
                    sold_qty = int(record.get("Vendita registrata", 0))
                    lordo = selling_price * sold_qty
                except ValueError:
                    print_colored(f"Errore di conversione per il prodotto {record['Prodotto']}", "red_bold")
                    continue
    
                profit_unit = selling_price - purchase_price
                profit_product = profit_unit * sold_qty
                profit_total += profit_product
                print_colored(f"Profitto : ", "blue_bold", end="")
                print_colored(f"{record['Prodotto']} ", "green_bold", end="")
                print_colored(f"lordo={lordo} netto= {profit_product:.2f} €", "blue_bold")
        
            print_colored(f"\nProfitto Totale: {profit_total:.2f} €", "blue_bold")
       
            continue

        elif value == "vendita":
            get_product_from_terminal_for_sale()
            
            
        else:
            print_colored("\nComando non valido", "red_bold")
            explain_commands()
            continue


I comandi disponibili sono i seguenti:
aggiungi: aggiungi un prodotto al magazzino
elenca: elenca i prodotti in magazzino
vendita: registra una vendita effettuata
profitti: mostra i profitti totali
aiuto: mostra i possibili comandi
chiudi: esci dal programma

Se digiti 'chiudi' per terminare il programma, i dati non ancora salvati andranno persi


Inserisci i dati dei prodotti da aggiungere o aggiornare nel magazzino




Inserisci un comando (aggiungi, elenca, vendita, profitti, aiuto, chiudi)



 aggiungi


Comando 'aggiungi' selezionato




NUOVO INSERIMENTO - N.1

Compila tutti i campi richiesti

Lascia il campo vuoto e Premi INVIO per terminare
I dati non ancora salvati andranno persi

Nome del prodotto: 

 cacao



Il prodotto esiste già. Inserisci solo la quantità da aggiungere:


...

CALCOLO DEI PROFITTI:

Profitto : alghe lordo=826.0 netto= 236.00 €
Profitto : avena in polvere lordo=348.5 netto= 114.75 €
Profitto : banane lordo=351.0 netto= 78.00 €
Profitto : barretta proteica lordo=35.0 netto= 15.40 €
Profitto : cacao lordo=13311.0 netto= 0.00 €
Profitto : crema di banana lordo=1050.0 netto= 350.00 €
Profitto : crema di mandorle lordo=32.5 netto= 13.50 €
Profitto : riso basmati lordo=120.0 netto= 30.00 €
Profitto : soia lordo=3.0 netto= 1.00 €

Profitto Totale: 838.65 €




Inserisci un comando (aggiungi, elenca, vendita, profitti, aiuto, chiudi)



 chiudi


Programma terminato per richiesta dell'utente


SystemExit: 

/home/gap/neural_env/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
